# Filtragem e Análise de Dados de Desmatamento
Este notebook realiza a filtragem e análise de dados de desmatamento para os municípios da Serra do Penitente.

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Criar diretórios de saída
os.makedirs("data/generated", exist_ok=True)
os.makedirs("results/figures", exist_ok=True)

## Carregar o arquivo de dados de desmatamento
Carregamos o arquivo CSV gerado pelo MapBiomas contendo os alertas de desmatamento.

In [ ]:
# Caminho do arquivo de entrada
input_path = "data/downloads/alertas_mapbiomas.csv"
df = pd.read_csv(input_path)
print("Colunas disponíveis:", df.columns.tolist())

## Identificar colunas relevantes
Identificamos automaticamente as colunas que contêm informações sobre município, ano e área desmatada.

In [ ]:
# Inferir nomes das colunas automaticamente
municipio_col = None
ano_col = None
area_col = None
for col in df.columns:
    if "municip" in col.lower():
        municipio_col = col
    if "ano" in col.lower() or "year" in col.lower():
        ano_col = col
    if "area" in col.lower() and ("ha" in col.lower() or "desmatada" in col.lower()):
        area_col = col
if not all([municipio_col, ano_col, area_col]):
    raise ValueError("Não foi possível identificar as colunas corretas. Verifique o CSV.")
print(f"Usando as colunas: Município = {municipio_col}, Ano = {ano_col}, Área Desmatada = {area_col}")

## Filtrar dados para os municípios de interesse
Selecionamos apenas os dados dos municípios da Serra do Penitente.

In [ ]:
# Definir municípios alvo
municipios_alvo = ["Balsas", "Tasso Fragoso", "Alto Parnaíba"]
df_filtrado = df[df[municipio_col].isin(municipios_alvo)]

## Agrupar dados por município e ano
Agrupamos os dados para calcular a área total desmatada por município e ano.

In [ ]:
# Agrupar por município e ano
df_agrupado = df_filtrado.groupby([municipio_col, ano_col])[area_col].sum().reset_index()
df_agrupado.columns = ["municipio", "ano", "area_desmatada_ha"]
print(df_agrupado.head())

## Salvar os dados filtrados
Salvamos os dados filtrados e agrupados em um novo arquivo CSV.

In [ ]:
# Salvar CSV
output_path = "data/generated/desmatamento_serra_penitente.csv"
df_agrupado.to_csv(output_path, index=False)
print(f"✅ Dataset salvo em: {output_path}")

## Gerar gráfico de evolução do desmatamento
Criamos um gráfico de linha para visualizar a evolução do desmatamento ao longo dos anos.

In [ ]:
# Gerar gráfico de linha
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_agrupado, x="ano", y="area_desmatada_ha", hue="municipio", marker="o")
plt.title("Evolução do Desmatamento nos Municípios da Serra do Penitente (2000-2023)")
plt.xlabel("Ano")
plt.ylabel("Área Desmatada (ha)")
plt.grid(True)
plt.legend(title="Município")
plt.tight_layout()
graph_output_path = "results/figures/evolucao_desmatamento_serra_penitente.png"
plt.savefig(graph_output_path)
plt.close()
print(f"✅ Gráfico salvo em: {graph_output_path}")